# COURSE CODE: CSC825
# COURSE TITLE: PROGRAMMING LANGUAGE
# STUDENT NAME: OLUWAKEMI OLUWATUNMISE OJO-AKINTAYO
# PG 202443229442


In [8]:


from abc import ABC, abstractmethod
from datetime import datetime
import uuid
from typing import List, Dict

class Person(ABC):
    """Abstract base class for all persons in the school system"""

    def __init__(self, name: str, email: str, phone: str):
        self._id = str(uuid.uuid4())[:8]  # ENCAPSULATION: Protected attribute
        self._name = name
        self._email = email
        self._phone = phone
        self._registration_date = datetime.now()

    # ENCAPSULATION: Using properties to control access to attributes
    @property
    def id(self):
        return self._id

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if value and isinstance(value, str):
            self._name = value

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, value):
        if value and isinstance(value, str):
            self._email = value

    @property
    def phone(self):
        return self._phone

    @phone.setter
    def phone(self, value):
        if value and isinstance(value, str):
            self._phone = value

    @property
    def registration_date(self):
        return self._registration_date

    # ABSTRACTION: Abstract methods to be implemented by subclasses
    @abstractmethod
    def get_role(self):
        """Return the role of the person in the school"""
        pass

    @abstractmethod
    def display_info(self):
        """Display complete information about the person"""
        pass

class Course(ABC):
    """Abstract base class for all courses in the school"""

    def __init__(self, course_code: str, course_name: str, credits: int):
        self._course_code = course_code
        self._course_name = course_name
        self._credits = credits
        self._max_students = 30  # ENCAPSULATION: Default capacity

    @property
    def course_code(self):
        return self._course_code

    @property
    def course_name(self):
        return self._course_name

    @property
    def credits(self):
        return self._credits

    @property
    def max_students(self):
        return self._max_students

    # ABSTRACTION: Abstract method for course-specific details
    @abstractmethod
    def get_course_type(self):
        """Return the type of course"""
        pass

    @abstractmethod
    def get_prerequisites(self):
        """Return course prerequisites"""
        pass

    @abstractmethod
    def display_course_info(self):
        """Display complete course information"""
        pass

# =============================================
# INHERITANCE: Person Subclasses
# =============================================

class Student(Person):
    """Student class inheriting from Person - represents a student in the school"""

    def __init__(self, name: str, email: str, phone: str, grade_level: str):
        super().__init__(name, email, phone)
        self._grade_level = grade_level
        self._enrolled_courses = []  # ENCAPSULATION: Private list
        self._gpa = 0.0
        self._total_credits = 0

    @property
    def grade_level(self):
        return self._grade_level

    @grade_level.setter
    def grade_level(self, value):
        if value and isinstance(value, str):
            self._grade_level = value

    @property
    def gpa(self):
        return self._gpa

    # POLYMORPHISM: Implementing abstract methods specifically for Student
    def get_role(self):
        return "Student"

    def display_info(self):
        info = f"Student ID: {self._id}\nName: {self._name}\nGrade: {self._grade_level}\n"
        info += f"Email: {self._email}\nPhone: {self._phone}\nGPA: {self._gpa:.2f}\nCredits: {self._total_credits}"
        return info

    # Student-specific methods
    def enroll_in_course(self, course) -> bool:
        """Enroll student in a course if there's space"""
        if not isinstance(course, Course):
            print("Error: Must provide a valid Course object")
            return False

        if len(self._enrolled_courses) >= 5:  # Max 5 courses per student
            print(f"Cannot enroll {self._name} in more than 5 courses")
            return False

        if course in self._enrolled_courses:
            print(f"{self._name} is already enrolled in {course.course_name}")
            return False

        self._enrolled_courses.append(course)
        print(f"Successfully enrolled {self._name} in {course.course_name}")
        return True

    def drop_course(self, course) -> bool:
        """Drop a course from student's schedule"""
        if course in self._enrolled_courses:
            self._enrolled_courses.remove(course)
            print(f"Successfully dropped {course.course_name} for {self._name}")
            return True
        print(f"{self._name} is not enrolled in {course.course_name}")
        return False

    def calculate_gpa(self, grades: Dict[str, float]) -> float:
        """Calculate GPA based on course grades"""
        if not grades:
            self._gpa = 0.0
            return self._gpa

        total_points = 0
        total_credits = 0

        for course_code, grade in grades.items():
            # Validate grade
            if not isinstance(grade, (int, float)) or grade < 0 or grade > 4.0:
                print(f"Warning: Invalid grade {grade} for course {course_code}")
                continue

            # Find course by code and get its credits
            course = next((c for c in self._enrolled_courses if c.course_code == course_code), None)
            if course:
                total_points += grade * course.credits
                total_credits += course.credits

        if total_credits > 0:
            self._gpa = total_points / total_credits
            self._total_credits = total_credits
        else:
            self._gpa = 0.0

        return round(self._gpa, 2)

    def get_enrolled_courses(self):
        """Get list of enrolled courses (ENCAPSULATION: returning copy)"""
        return self._enrolled_courses.copy()

    def get_course_count(self):
        """Get number of enrolled courses"""
        return len(self._enrolled_courses)

class Teacher(Person):
    """Teacher class inheriting from Person - represents a teacher in the school"""

    def __init__(self, name: str, email: str, phone: str, department: str):
        super().__init__(name, email, phone)
        self._department = department
        self._assigned_courses = []  # ENCAPSULATION: Private list

    @property
    def department(self):
        return self._department

    @department.setter
    def department(self, value):
        if value and isinstance(value, str):
            self._department = value

    # POLYMORPHISM: Implementing abstract methods specifically for Teacher
    def get_role(self):
        return "Teacher"

    def display_info(self):
        info = f"Teacher ID: {self._id}\nName: {self._name}\nDepartment: {self._department}\n"
        info += f"Email: {self._email}\nPhone: {self._phone}\nCourses Assigned: {len(self._assigned_courses)}"
        return info

    # Teacher-specific methods
    def assign_course(self, course) -> bool:
        """Assign a course to the teacher"""
        if not isinstance(course, Course):
            print("Error: Must provide a valid Course object")
            return False

        if course not in self._assigned_courses:
            self._assigned_courses.append(course)
            print(f"Assigned {course.course_name} to {self._name}")
            return True
        print(f"{self._name} is already assigned to {course.course_name}")
        return False

    def get_assigned_courses(self):
        """Get list of assigned courses"""
        return self._assigned_courses.copy()

# =============================================
# INHERITANCE: Course Subclasses
# =============================================

class CoreCourse(Course):
    """CoreCourse class inheriting from Course - represents required courses"""

    def __init__(self, course_code: str, course_name: str, credits: int, department: str):
        super().__init__(course_code, course_name, credits)
        self._department = department
        self._is_required = True

    # POLYMORPHISM: Implementing abstract methods specifically for CoreCourse
    def get_course_type(self):
        return "Core Course"

    def get_prerequisites(self):
        return "No prerequisites - Required course"

    def display_course_info(self):
        info = f"{self._course_code}: {self._course_name}\n"
        info += f"Type: {self.get_course_type()}\nCredits: {self._credits}\n"
        info += f"Department: {self._department}\nRequired: {self._is_required}\n"
        info += f"Max Students: {self._max_students}"
        return info

class ElectiveCourse(Course):
    """ElectiveCourse class inheriting from Course - represents optional courses"""

    def __init__(self, course_code: str, course_name: str, credits: int, department: str, prerequisites: List[str]):
        super().__init__(course_code, course_name, credits)
        self._department = department
        self._prerequisites = prerequisites
        self._is_required = False

    # POLYMORPHISM: Implementing abstract methods specifically for ElectiveCourse
    def get_course_type(self):
        return "Elective Course"

    def get_prerequisites(self):
        if self._prerequisites:
            return f"Prerequisites: {', '.join(self._prerequisites)}"
        return "No prerequisites"

    def display_course_info(self):
        info = f"{self._course_code}: {self._course_name}\n"
        info += f"Type: {self.get_course_type()}\nCredits: {self._credits}\n"
        info += f"Department: {self._department}\nRequired: {self._is_required}\n"
        info += f"Max Students: {self._max_students}\n"
        info += f"{self.get_prerequisites()}"
        return info

# =============================================
# COMPOSITION: School Management System
# =============================================

class SchoolRegistrationPortal:
    """Main school registration system that composes all other classes"""

    def __init__(self, school_name: str):
        self._school_name = school_name
        self._students = {}  # ENCAPSULATION: Private dictionaries
        self._teachers = {}
        self._courses = {}
        self._academic_year = "2024-2025"

    # Student management methods
    def register_student(self, name: str, email: str, phone: str, grade_level: str) -> str:
        """Register a new student and return student ID"""
        student = Student(name, email, phone, grade_level)
        self._students[student.id] = student
        print(f"Successfully registered student: {name} (ID: {student.id})")
        return student.id

    def enroll_student_in_course(self, student_id: str, course_code: str) -> bool:
        """Enroll a student in a specific course"""
        student = self._students.get(student_id)
        course = self._courses.get(course_code)

        if not student or not course:
            print("Student or course not found")
            return False

        # Check if course is full
        enrolled_count = sum(1 for s in self._students.values()
                           if course in s.get_enrolled_courses())

        if enrolled_count >= course.max_students:
            print(f"Course {course.course_name} is full (max {course.max_students} students)")
            return False

        return student.enroll_in_course(course)

    # Teacher management methods
    def hire_teacher(self, name: str, email: str, phone: str, department: str) -> str:
        """Hire a new teacher and return teacher ID"""
        teacher = Teacher(name, email, phone, department)
        self._teachers[teacher.id] = teacher
        print(f"Successfully hired teacher: {name} (ID: {teacher.id})")
        return teacher.id

    def assign_teacher_to_course(self, teacher_id: str, course_code: str) -> bool:
        """Assign a teacher to a specific course"""
        teacher = self._teachers.get(teacher_id)
        course = self._courses.get(course_code)

        if teacher and course:
            return teacher.assign_course(course)
        else:
            print("Teacher or course not found")
            return False

    # Course management methods
    def add_course(self, course_type: str, **kwargs) -> str:
        """Add a new course to the system"""
        course = None

        # POLYMORPHISM: Creating different course types through common interface
        if course_type.lower() == "core":
            course = CoreCourse(
                kwargs['course_code'],
                kwargs['course_name'],
                kwargs['credits'],
                kwargs['department']
            )
        elif course_type.lower() == "elective":
            course = ElectiveCourse(
                kwargs['course_code'],
                kwargs['course_name'],
                kwargs['credits'],
                kwargs['department'],
                kwargs.get('prerequisites', [])
            )
        else:
            raise ValueError("Invalid course type")

        self._courses[course.course_code] = course
        print(f"Successfully added course: {course.course_name} ({course.course_code})")
        return course.course_code

    # Reporting methods
    def generate_student_report(self, student_id: str):
        """Generate a comprehensive report for a student"""
        student = self._students.get(student_id)
        if not student:
            print("Student not found")
            return

        print(f"\n{'='*50}")
        print(f"STUDENT REPORT - {self._school_name}")
        print(f"{'='*50}")
        print(student.display_info())
        print(f"\nEnrolled Courses ({student.get_course_count()}):")
        for course in student.get_enrolled_courses():
            print(f"  - {course.course_code}: {course.course_name} ({course.credits} credits)")
        print(f"{'='*50}")

    def generate_teacher_report(self, teacher_id: str):
        """Generate a report for a teacher"""
        teacher = self._teachers.get(teacher_id)
        if not teacher:
            print("Teacher not found")
            return

        print(f"\n{'='*50}")
        print(f"TEACHER REPORT - {self._school_name}")
        print(f"{'='*50}")
        print(teacher.display_info())
        print(f"\nAssigned Courses ({len(teacher.get_assigned_courses())}):")
        for course in teacher.get_assigned_courses():
            print(f"  - {course.course_code}: {course.course_name}")
        print(f"{'='*50}")

    def display_all_courses(self):
        """Display all available courses"""
        print(f"\n{'='*60}")
        print(f"AVAILABLE COURSES - {self._school_name} ({self._academic_year})")
        print(f"{'='*60}")
        for course_code, course in self._courses.items():
            print(course.display_course_info())
            print("-" * 40)

    def get_student_count(self):
        """Get total number of students"""
        return len(self._students)

    def get_teacher_count(self):
        """Get total number of teachers"""
        return len(self._teachers)

    def get_course_count(self):
        """Get total number of courses"""
        return len(self._courses)

# =============================================
# DEMONSTRATION: Showing OOP Features in Action
# =============================================

def main():
    """Main function to demonstrate the School Registration Portal"""

    # Create school registration portal
    school = SchoolRegistrationPortal("Python High School")

    print("=== SCHOOL REGISTRATION PORTAL DEMONSTRATION ===")
    print("Showing all four OOP pillars: Encapsulation, Inheritance, Polymorphism, and Abstraction\n")

    # ==================================================================
    # DEMONSTRATION 1: COURSE CREATION (POLYMORPHISM & INHERITANCE)
    # ==================================================================
    print("1. CREATING COURSES (POLYMORPHISM & INHERITANCE):")
    print("-" * 40)

    # Create core courses
    school.add_course("core",
                     course_code="CSC825",
                     course_name="Programming Language",
                     credits=4,
                     department="Computer Science")

    school.add_course("core",
                     course_code="CSC803",
                     course_name="Advanced Computer Algorithm",
                     credits=3,
                     department="Computer Science")

    # Create elective courses with prerequisites
    school.add_course("elective",
                     course_code="CSC815",
                     course_name="Operation Research",
                     credits=4,
                     department="Computer Science",
                     prerequisites=["CSC825", "CSC803"])

    school.add_course("elective",
                     course_code="CSC871",
                     course_name="Research Methodology",
                     credits=2,
                     department="Science",
                     prerequisites=[])

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 2: STUDENT REGISTRATION (ENCAPSULATION)
    # ==================================================================
    print("2. STUDENT REGISTRATION (ENCAPSULATION):")
    print("-" * 40)

    # Register students with your name
    kemi_id = school.register_student("Oluwakemi Ojo-Akintayo", "oluwakemiojo@gmail.com", "555-0101", "MSC")
    dayo_id = school.register_student("Dayo Siobaloju", "dayosiobaloju@email.com", "555-0102", "PGD")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 3: TEACHER HIRING (INHERITANCE)
    # ==================================================================
    print("3. TEACHER HIRING (INHERITANCE):")
    print("-" * 40)

    # Hire teachers
    mr_wilson_id = school.hire_teacher("Mr. Wilson", "wilson@school.edu", "555-0201", "Mathematics")
    ms_davis_id = school.hire_teacher("Ms. Davis", "davis@school.edu", "555-0202", "Computer Science")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 4: COURSE ASSIGNMENTS (POLYMORPHISM)
    # ==================================================================
    print("4. COURSE ASSIGNMENTS (POLYMORPHISM):")
    print("-" * 40)

    # Assign teachers to courses
    school.assign_teacher_to_course(mr_wilson_id, "CSC825")
    school.assign_teacher_to_course(ms_davis_id, "CSC815")

    # Enroll students in courses
    school.enroll_student_in_course(kemi_id, "CSC825")
    school.enroll_student_in_course(kemi_id, "CSC803")
    school.enroll_student_in_course(kemi_id, "CSC871")

    school.enroll_student_in_course(dayo_id, "CSC825")
    school.enroll_student_in_course(dayo_id, "CSC815")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 5: GPA CALCULATION & REPORTING (ABSTRACTION)
    # ==================================================================
    print("5. GPA CALCULATION & REPORTING (ABSTRACTION):")
    print("-" * 40)

    # Get student objects for direct operations
    kemi = school._students[kemi_id]  # Accessing protected member for demo
    dayo = school._students[dayo_id]

    # Calculate GPA for students (simulated grades)
    kemi_grades = {"CSC825": 3.8, "CSC803": 3.5, "CSC871": 4.0}
    dayo_grades = {"CSC825": 3.2, "CSC815": 3.7}

    kemi.calculate_gpa(kemi_grades)
    dayo.calculate_gpa(dayo_grades)

    print(f"Oluwakemi's GPA: {kemi.gpa:.2f}")
    print(f"Dayo's GPA: {dayo.gpa:.2f}")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 6: COMPREHENSIVE REPORTS
    # ==================================================================
    print("6. COMPREHENSIVE REPORTS:")
    print("-" * 40)

    # Generate student reports
    school.generate_student_report(kemi_id)
    school.generate_student_report(dayo_id)

    # Generate teacher report
    school.generate_teacher_report(ms_davis_id)

    # Display all available courses
    school.display_all_courses()

    # ==================================================================
    # DEMONSTRATION 7: ERROR HANDLING & VALIDATION (ENCAPSULATION)
    # ==================================================================
    print("\n7. ERROR HANDLING & VALIDATION (ENCAPSULATION):")
    print("-" * 40)

    # Try to enroll in non-existent course
    school.enroll_student_in_course(kemi_id, "PHYSICS101")

    # Try to assign non-existent teacher
    school.assign_teacher_to_course("INVALID_ID", "CSC825")

    # Try to enroll in too many courses
    school.enroll_student_in_course(kemi_id, "CSC815")  # Should fail - already at max

    # ==================================================================
    # DEMONSTRATION 8: SYSTEM STATISTICS
    # ==================================================================
    print("\n8. SYSTEM STATISTICS:")
    print("-" * 40)
    print(f"Total Students: {school.get_student_count()}")
    print(f"Total Teachers: {school.get_teacher_count()}")
    print(f"Total Courses: {school.get_course_count()}")

if __name__ == "__main__":
    main()

=== SCHOOL REGISTRATION PORTAL DEMONSTRATION ===
Showing all four OOP pillars: Encapsulation, Inheritance, Polymorphism, and Abstraction

1. CREATING COURSES (POLYMORPHISM & INHERITANCE):
----------------------------------------
Successfully added course: Programming Language (CSC825)
Successfully added course: Advanced Computer Algorithm (CSC803)
Successfully added course: Operation Research (CSC815)
Successfully added course: Research Methodology (CSC871)


2. STUDENT REGISTRATION (ENCAPSULATION):
----------------------------------------
Successfully registered student: Oluwakemi Ojo-Akintayo (ID: 99671300)
Successfully registered student: Dayo Siobaloju (ID: 335e4be9)


3. TEACHER HIRING (INHERITANCE):
----------------------------------------
Successfully hired teacher: Mr. Wilson (ID: 40ee692a)
Successfully hired teacher: Ms. Davis (ID: 18831f97)


4. COURSE ASSIGNMENTS (POLYMORPHISM):
----------------------------------------
Assigned Programming Language to Mr. Wilson
Assigned Ope